# agentscope.message

agentsope.message 定义了结构化消息模块（Msg）及其内容块（TextBlock, ThinkingBlock, Base64Source, URLSource, ImageBlock, AudioBlock, VideoBlock, ToolUseBlock, ToolResultBlock）。用于在多模态、工具调用等复杂智能体（Agent）交互中统一消息格式。

## Class Msg
    消息主体，支持两种内容格式：
1. 纯文本字符串
2. 结构化内容块列表（多模态和工具调用）

初始化参数：
### __init__(name, content, role, metadata=None, timestamp=None, invocation_id=None)
Initialize the Msg object.

参数:
- name (str) -- 发送者名称

- content (str | list[ContentBlock]) -- 发送内容：字符串或字符串列表

- role (Literal["user", "assistant", "system"]) -- 角色

- metadata (dict[str, JSONSerializableObject] | None, optional) -- 附加的结构化数据

- timestamp (str | None, optional) -- 时间戳

- invocation_id (str | None, optional) -- API调用ID，用于追踪

返回类型:
None

### to_dict()
Convert the message into JSON dict data.

返回类型:
dict

### classmethod from_dict(json_data)
Load a message object from the given JSON data.

参数:
json_data (dict)

返回类型:
Msg

### has_content_blocks(block_type=None)
Check if the message has content blocks of the given type.

参数:
block_type (Literal["text", "tool_use", "tool_result", "image", "audio", "video"] | None, defaults to None) -- The type of the block to be checked. If None, it will check if there are any content blocks.

返回类型:
bool

### get_text_content()
Get the pure text blocks from the message content.

返回类型:
str | None

### get_content_blocks(block_type)
get_content_blocks(block_type: Literal['text']) → List[TextBlock]
get_content_blocks(block_type: Literal['tool_use']) → List[ToolUseBlock]
get_content_blocks(block_type: Literal['tool_result']) → List[ToolResultBlock]
get_content_blocks(block_type: Literal['image']) → List[ImageBlock]
get_content_blocks(block_type: Literal['audio']) → List[AudioBlock]
get_content_blocks(block_type: Literal['video']) → List[VideoBlock]
get_content_blocks(block_type: None = None) → List[ToolUseBlock | ToolResultBlock | TextBlock | ThinkingBlock | ImageBlock | AudioBlock | VideoBlock]
Get the content in block format. If the content is a string, it will be converted to a text block.

参数:
block_type (Literal["text", "thinking", "tool_use", "tool_result", "image", "audio", "video"] | None, optional) -- The type of the block to be extracted. If None, all blocks will be returned.

返回:
The content blocks.

返回类型:
List[ContentBlock]

In [1]:
# 纯文本消息
from agentscope.message import Msg

msg = Msg(
    name="user",
    content="你好，请帮我查天气。",
    role="user"
)
print(msg.content)  # 输出: "你好，请帮我查天气。"

你好，请帮我查天气。


In [ ]:
# 多模态+工具调用消息
from agentscope.message import Msg, TextBlock, ToolUseBlock, ImageBlock, URLSource

# 构造内容块
blocks = [
    TextBlock(type="text", text="我需要查询北京天气，并附上一张地图。"),
    ImageBlock(type="image", source=URLSource(type="url", url="https://picsum.photos/400/400?random=1")),
    ToolUseBlock(
        type="tool_use",
        id="tool_001",
        name="get_weather",
        input={"location": "Beijing"}
    )
    # 其他的查找官方API文档给的参数，操作类似
]

msg = Msg(
    name="user",
    content=blocks,
    role="user"
)

# 检查是否包含工具调用
print(msg.has_content_blocks("tool_use"))  # True

# 提取工具调用块
tool_calls = msg.get_content_blocks("tool_use")
print(tool_calls[0]["name"])  # "get_weather"

True
get_weather


In [10]:
# ImageBlock （支持Base64和URL）
from agentscope.message import ImageBlock, URLSource, Base64Source
# 方式1： URL
ImageBlock(source=URLSource(url="https://picsum.photos/400/400?random=1"))

{'source': {'url': 'https://picsum.photos/400/400?random=1'}}

In [11]:
# 方式2：Base64
ImageBlock(source=Base64Source(
    media_type="image/jpeg",
    data="/9j/4AAQSkZJRgABAQE..."
))

{'source': {'media_type': 'image/jpeg', 'data': '/9j/4AAQSkZJRgABAQE...'}}

In [13]:
from agentscope.message import ToolResultBlock

ToolResultBlock(
    id="call_123",  # 必须与 ToolUseBlock.id 匹配
    name="search_web",
    output="AgentScope v0.2.0 已发布..."
)
# output 也可以是多模态内容：
ToolResultBlock(
    id="call_img",
    name="generate_image",
    output=[
        ImageBlock(source=URLSource(url="https://picsum.photos/400/400?random=1"))
    ]
)

{'id': 'call_img',
 'name': 'generate_image',
 'output': [{'source': {'url': 'https://picsum.photos/400/400?random=1'}}]}